In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import time
from google.colab import drive

In [2]:

#1. Configuracion
drive.mount('/content/drive')

DATA_DIR = '/content/drive/MyDrive/Master Thesis/Data/Split_Dataset'
Save_Path = '/content/drive/MyDrive/Master Thesis/Models/vit_archivtecture_model.pth'

#Crear carpeta de modelos si no existe
os.makedirs(os.path.dirname(Save_Path), exist_ok=True)

#Configuracion de dispositivos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Ejectuando en: {device}")


Mounted at /content/drive
Ejectuando en: cuda


In [3]:

#2. Transformaciones (Data Augmentation para Arquitectura)
data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [4]:

#3. Carga de Datos
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                  for x in ['train', 'val']}

data_loaders = {x: DataLoader(image_datasets[x], batch_size= 32, shuffle=True, num_workers=2, pin_memory=True)
                  for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
num_classes = len(class_names)
print(f"Dataset cargado: {num_classes} monumentos identificados.")


Dataset cargado: 14 monumentos identificados.


In [5]:

#4. Carga deñ Vosopm Transformer (ViT-B/16)
#Usamos weights=ViT_B_16_Weights.IMAGENET1K_V1 para Transfer Learning
model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)

#Ajustamos la cabeza (head) para nuestras clases de arquitectura
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)
model = model.to(device)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:02<00:00, 125MB/s]


In [6]:

#5. Funcion de Perdida y Optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


In [7]:
#6. Bucle de entrenamiento
print("Iniciando entrenamiento...")

epochs = 15
best_acc = 0.0

for epoch in range(epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in data_loaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_acc = running_corrects.double() / len(image_datasets[phase])
        print(f"Epoca {epoch} | {phase} Acc: {epoch_acc:.4}")

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), Save_Path)
            print("Modelo guardado!")

print("Entrenamiento finalizado!")


Iniciando entrenamiento...
Epoca 0 | train Acc: 0.8431
Epoca 0 | val Acc: 0.9127
Modelo guardado!
Epoca 1 | train Acc: 0.9503
Epoca 1 | val Acc: 0.8994
Epoca 2 | train Acc: 0.9624
Epoca 2 | val Acc: 0.9283
Modelo guardado!
Epoca 3 | train Acc: 0.9751
Epoca 3 | val Acc: 0.8891
Epoca 4 | train Acc: 0.978
Epoca 4 | val Acc: 0.8972
Epoca 5 | train Acc: 0.9845
Epoca 5 | val Acc: 0.8743
Epoca 6 | train Acc: 0.9792
Epoca 6 | val Acc: 0.8883
Epoca 7 | train Acc: 0.9777
Epoca 7 | val Acc: 0.912
Epoca 8 | train Acc: 0.989
Epoca 8 | val Acc: 0.9112
Epoca 9 | train Acc: 0.9836
Epoca 9 | val Acc: 0.9009
Epoca 10 | train Acc: 0.9855
Epoca 10 | val Acc: 0.895
Epoca 11 | train Acc: 0.9754
Epoca 11 | val Acc: 0.8676
Epoca 12 | train Acc: 0.9907
Epoca 12 | val Acc: 0.9253
Epoca 13 | train Acc: 0.9914
Epoca 13 | val Acc: 0.8972
Epoca 14 | train Acc: 0.9873
Epoca 14 | val Acc: 0.9098
Entrenamiento finalizado!


In [8]:
#7. Validacion de archivos (0-Bytes)
if os.path.exists(Save_Path) and os.path.getsize(Save_Path)>0:
    print(f"Fase 2 completada. Modelo guardado: {Save_Path}")
else:
    print("Error de validacion, archivo de 0 bytes o inexistente")

Fase 2 completada. Modelo guardado: /content/drive/MyDrive/Master Thesis/Models/vit_archivtecture_model.pth
